# config the workspace

In [ ]:
from Lab_Equipment.Config import config 

ModuleNotFoundError: No module named 'Lab_Equipment'

: 

# load all libs

In [ ]:
# Python Libs
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ctypes
import copy
from IPython.display import display, clear_output
import ipywidgets
import multiprocessing
from multiprocessing import shared_memory
import time
import scipy.io

from scipy import io, integrate, linalg, signal
from scipy.io import savemat, loadmat
from scipy.fft import fft, fftfreq, fftshift,ifftshift, fft2,ifft2,rfft2,irfft2
# Defult Pploting properties 
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [5,5]

#General libs
import Lab_Equipment.GeneralLibs.ComplexPlotFunction as cmplxplt

#Laser Lib
import Lab_Equipment.Laser.SantecSweeptLaser as laserlib
#OSA Lib
import  Lab_Equipment.OSA.OSA_Yokogawa_lib as OSAlib
#powerMeter
import Lab_Equipment.PowerMeter.PowerMeterObject as pwrMetlib

import Lab_Equipment.Laser.NKTPhotonics_Laser as NKTLaserLib

# timeTagger
import TimeTagger
import Lab_Equipment.TimeTagger.TimeTaggerInterface as TimeTaggerMod
import Lab_Equipment.TimeTagger.TimeTaggerFunction as TimetaggerFunc
# import Lab_Equipment.TimeTagger.TimeTaggerCustomMeasurementObj as TimeTaggerCustomObj


In [6]:
 
# This is needed to Qt stuff (I think()
# %pylab inline
# %gui qt5

%load_ext autoreload
#Config File
%aimport Lab_Equipment.Config.config
#General libs
%aimport Lab_Equipment.GeneralLibs.ComplexPlotFunction
#Laser libs
%aimport Lab_Equipment.Laser.SantecSweeptLaser
%aimport Lab_Equipment.Laser.NKTPhotonics_Laser
#OSA Libs
%aimport Lab_Equipment.OSA.OSA_Yokogawa_lib


# timeTagger
%aimport Lab_Equipment.TimeTagger.TimeTaggerInterface
%aimport Lab_Equipment.TimeTagger.TimeTaggerFunction
# %aimport Lab_Equipment.TimeTagger.TimeTaggerCustomMeasurementObj


%autoreload 1


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# NKT Laser

In [ ]:
NKTlaser=NKTLaserLib.Laser_NKT_BasiK_K1x2('COM5')
#call  del NKTlaser to disconnect the laser
# NKTlaser.Get_Power()
NKTlaser.Set_Power(12) #Input units are mW 
# NKTlaser.Get_Wavelength()
# NKTlaser.Set_Wavelength(1550.11) #Input units are nm 

Opening the comport: 1:OPFailed
Emission ON - Result: 0
Units set to mW
Input/output units for Set_Wavelengh()/Get_Wavelength() are nm 
Input/output units for Set_Power()/Get_Power() are mW 
Module Setup Bits:
- Narrow wavelength modulation range: Disabled
- Enable external wavelength modulation: Enabled
- Wavelength modulation DC coupled: Disabled
- Enable internal wavelength modulation: Disabled
- Enable modulation output: Disabled
- Pump operation constant current: Disabled
- External amplitude modulation source: Disabled
My creator got annoyed at NKT and has not immplemented enabling or disabling these features yet


12.05

# Power meter

In [ ]:

pwrMeter=pwrMetlib.PowerMeterObj('USB0::0x1313::0x8076::M01230617::INSTR',wavelength=1550,AvgCount=10)
#call  del pwrMeter to disconnect the power meter
pwrMeter.SetWaveLength(1550)# units are in nm
pwrMeter.GetPower() #units are in Watts

# Swept laser

In [ ]:
SweeptLaser=laserlib.LaserObject_SantecSweept('GPIB0::2::INSTR')
#call  del SweeptLaser to disconnect the Swept laser
# SweeptLaser.Set_wavelength(1550) #units are in nm
# SweeptLaser.Set_PowerLevel(0.5) #units are in mW



# OSA

In [ ]:
OSA=OSAlib.OSA_Yoko('GPIB0::1::INSTR')
# call del OSA to disconnect the OSA
#OSA.Initiate_Sweep("AUTO")
# Get some data from the OSA. A inital Sweep should be run on AUTO i.e. OSA.Initiate_Sweep("AUTO")
# so that all the properties of the OSA are set up automatically
# Wavelength,PowerSpectum=OSA.Get_data(TraceChannel="TRA",PowerDensityOutput=0,SweepMode="SINGle",Plotdata=True,)


Connection successful. Device ID: YOKOGAWA,AQ6374,91UC09551,01.02



# Time Tagger

In [ ]:
tagger = TimeTagger.createTimeTagger()

### Set global properties of the time tagger object

In [ ]:

#turn on the conditional filter
tagger.setConditionalFilter(trigger=[1,2], filtered=[4])
#tagger.setConditionalFilter(trigger=[1,2,3], filtered=[4])
#Turn off the conditional filter
# tagger.clearConditionalFilter()
sm = TimeTagger.SynchronizedMeasurements(tagger)
tagger.setInputDelay(1, 970)
tagger.setInputDelay(2, 900)
#tagger.setInputDelay(3, 2650)
tagger.setInputDelay(4, 6200)

In [ ]:
binCount=3000
channels=[1,2,3]
#channels=[1,2,3]
# ChannelCount=len(channels)
corr_list = list()
for ch in channels:
    corr_list.append(TimeTagger.Correlation(sm.getTagger(), channel_1=4, channel_2=ch, binwidth=1, n_bins=binCount))


corr_list.append(TimeTagger.Correlation(sm.getTagger(), channel_1=channels[0], channel_2=channels[1], binwidth=1, n_bins=binCount))
corr_list.append(TimeTagger.Correlation(sm.getTagger(), channel_1=channels[0], channel_2=channels[2], binwidth=1, n_bins=binCount))
corr_list.append(TimeTagger.Correlation(sm.getTagger(), channel_1=channels[1], channel_2=channels[2], binwidth=1, n_bins=binCount))


    
# for ch in channels:   
#     corr_list.append(TimeTaggerCustomObj.CustomCorrelationMeasurement(sm.getTagger(),[4], [ch], binwidth=1, n_bins=binCount,MeasurementType=1))
  
corr_list.append(TimeTaggerCustomObj.CustomCorrelationMeasurement(sm.getTagger(),[4], channels, binwidth=1, n_bins=binCount,MeasurementType=100))
# corr_list.append(TimeTagger.FileWriter(sm.getTagger(),'dump1234.ttbin',[1,2,3,4]))
# ChannelCountTotal=len(corr_list)
ChannelCount=len(corr_list)

## Set global properties of the time tagger object 

In [ ]:
# tagger.setInputDelay(3, 0)
tagger.setInputDelay(1, 6220-(700-81+35))
tagger.setInputDelay(2, 6220-700-90)
#tagger.setInputDelay(1, 2150-586-6-6+30+6)
#tagger.setInputDelay(2, 1800-317+12-19+4+10)
#tagger.setInputDelay(3, 1325-302+3)
tagger.setInputDelay(4, 0)
CountTime=40
timeData,CorrData, CorrDataNorm=  TimetaggerFunc.GetCorrelationdata(sm,corr_list,CountTime,3000,PlotResutls=True)

# If you want to get the time delay you can use either of the lines below
# TimetaggerFunc.CalculateTimeDelay(timeData,CorrDataNorm)
AvgTimeDelay_0,PeakTimeDelay_0 = TimetaggerFunc.CalculateTimeDelay(timeData[0,:],CorrDataNorm[0,:])
print(AvgTimeDelay_0,PeakTimeDelay_0)
AvgTimeDelay_1,PeakTimeDelay_1 =TimetaggerFunc.CalculateTimeDelay(timeData[0,:],CorrDataNorm[1,:])
print(AvgTimeDelay_1,PeakTimeDelay_1)
#AvgTimeDelay_2,PeakTimeDelay_2 =TimetaggerFunc.CalculateTimeDelay(timeData[0,:],CorrDataNorm[2,:])
#print(AvgTimeDelay_2,PeakTimeDelay_2)

# Do Experiment

In [ ]:
PowerLevelCount=10
PowerLevelMin=12
PowerLevelMax=40
PowerLevelSet_arr=np.linspace(PowerLevelMin,PowerLevelMax,PowerLevelCount)
PowerLevelAct_laser_arr=np.zeros((PowerLevelCount))
PowerLevelAct_afterChip_arr=np.zeros((PowerLevelCount))
PowerLevelAct_beforeChip_arr=np.zeros((PowerLevelCount))





In [ ]:
for i in range(PowerLevelCount):
    PowerLevelAct_laser_arr[i]=NKTlaser.Set_Power(PowerLevelSet_arr[i])
    PowerLevelAct_beforeChip_arr[i]=pwrMeter.GetPower()
    PowerLevelAct_afterChip_arr[i]=pwrMeter.GetPower()
    

In [ ]:
# Setup distance array for peizo mount
CountTime=40
maxdist=4 # this is in micros
mindist=0 # this is in micros
distSpacing_ideal =0.1 #can go as low as 0.01
distCount=int((maxdist-mindist)/distSpacing_ideal)
distArr_ideal=np.linspace(mindist,maxdist,distCount)
distSpacing=distArr_ideal[1]-distArr_ideal[0]
print(distCount,distSpacing_ideal,distSpacing)
# distCount=10

#Set mount to zero
PiezoMount.SetPosition(0.0001)

# make all the arrays for the data
timeDataAll=np.zeros((ChannelCount,binCount,distCount))
CorrDataAll=np.zeros((ChannelCount,binCount,distCount))
CorrDataNormAll=np.zeros((ChannelCount,binCount,distCount))
CorrDataAdd=np.zeros((binCount,distCount))
CorrDataNormAdd=np.zeros((binCount,distCount))
distArr_measured=np.zeros((distCount))
PlotDuringMeasument=True

if (PlotDuringMeasument):
    plt.figure(1,figsize=(4,4))
    
for idist in range(distCount):
    
    #################################
    #take correlations measurment
    #################################
    timeData , CorrData, CorrDataNorm = TimetaggerFunc.GetCorrelationdata(sm,corr_list,CountTime,binCount)
    timeDataAll[:,:,idist]=timeData
    CorrDataAll[:,:,idist]= CorrData
    CorrDataNormAll[:,:,idist]=CorrDataNorm
    CorrDataAdd[:,idist]=np.sum(CorrData,axis=0)
    CorrDataNormAdd[:,idist]=np.sum(CorrDataNorm,axis=0)
    
    #################################
    # Move the the BeamSplitter
    #################################
    if (idist==0):
        distArr_measured[idist]=float(str(PiezoMount.GetPosition()))
        print(distArr_measured[idist],distArr_ideal[idist])
    else:
       
        distArr_measured[idist]=float(str(PiezoMount.SetPosition(distArr_ideal[idist])))
        print(distArr_measured[idist],distArr_ideal[idist])
        print(PiezoMount.GetPosition())
        
    #################################
    #save the data While we are going. This is really ineffiecent but dont really care at the moment
    #################################
    np.savez_compressed('Data\\batch_data.npz',
                        timeDataAll=timeDataAll, CorrDataAll=CorrDataAll
                        ,CorrDataNormAll=CorrDataNormAll
                        ,CorrDataAdd=CorrDataAdd
                        ,CorrDataNormAdd=CorrDataNormAdd
                        ,distArr_ideal=distArr_ideal
                        ,distArr_measured=distArr_measured
                        ,AngleOfFirstWaveplate=Mount1Angle
                        ,AngleOfSecondWaveplate=Mount2Angle
                        )
    TimetaggerFunc.MoveRawTags(config.PATH_TO_TIMETAGGER_FOLDER+"Data\\RawTimeTags","Data\\RawTags\\RawtagMountPos"+str(idist)+"\\")
    # save the data So far using pickle. I think npz is faster
    # with open('Data\\batch_data.pkl', 'wb') as f:
    #     pickle.dump([timeDataAll, CorrDataAll,CorrDataNormAll,CorrDataAdd,CorrDataNormAdd], f)
    
    #################################
    # plot the result while we are giong
    #################################
    if (PlotDuringMeasument):
        plt.subplot(1,2,1)
        plt.imshow(CorrDataAdd,aspect='auto')
        plt.subplot(1,2,2)
        plt.imshow(CorrDataNormAdd,aspect='auto')
        plt.show
        
# 
